# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [84]:
import requests
import pandas as pd

# Load the bike station data from the CSV file
df_new = pd.read_csv('citybikes_data.csv')

In [88]:
import requests
import pandas as pd
import os
# Foursquare API details
api_key = "YOUR API KEY"

# Initialize an empty list to store results
result = []

# Iterate through the rows of the DataFrame
for index, row in df_new.iterrows():
    ll = row['ll']
    radius = 1000  # You can adjust the radius as needed
    categories = 13000  # You can adjust the category as needed
    limit = 50  # You can adjust the limit as needed

    URL = f"https://api.foursquare.com/v3/places/search?ll={ll}&radius={radius}&categories={categories}&sort=RATING&limit={limit}"

    headers = {
        "accept": "application/json",
        "Authorization": api_key
    }
    # make the API request
    response = requests.get(URL, headers=headers)
     data = response.json()  # Store result in data

In [ ]:
data

In [90]:
# Convert the dictionary into a DataFrame
data_dict = pd.DataFrame.from_dict(data, orient='index')

# Display the resulting DataFrame
print(data_dict)

                                              results     context
0   {'fsq_id': '4ef1daed0e617e29c65f76c9', 'catego...  geo_bounds
1   {'fsq_id': '4c595eb067ac0f474df7034c', 'catego...        None
2   {'fsq_id': '4b6cb501f964a520de4d2ce3', 'catego...        None
3   {'fsq_id': '4bf728165ec320a1b28d86d3', 'catego...        None
4   {'fsq_id': '4b9e2a28f964a520a6ce36e3', 'catego...        None
5   {'fsq_id': '4de9b9701f6e3ddebdba024f', 'catego...        None
6   {'fsq_id': '4c76ea765cd6b1f7d3743d94', 'catego...        None
7   {'fsq_id': '4bad451bf964a5203a403be3', 'catego...        None
8   {'fsq_id': '4e544e9fae60e3d6b7eccc42', 'catego...        None
9   {'fsq_id': '4d8ba6940c4e41bd15fa667f', 'catego...        None
10  {'fsq_id': '4d9d1dc37865a093ef80e17a', 'catego...        None
11  {'fsq_id': '4dd0499d52b1f8915bc47e93', 'catego...        None
12  {'fsq_id': '4bd4cfd329eb9c74bd7092e1', 'catego...        None
13  {'fsq_id': '5b66680d12c8f0002c524fb1', 'catego...        None
14  {'fsq_

,results,context
0,"{'fsq_id': '4ef1daed0e617e29c65f76c9', 'catego...",geo_bounds
1,"{'fsq_id': '4c595eb067ac0f474df7034c', 'catego...",None


In [ ]:
# SLice the dataframe and save in a new variable to further analyze the structure of the response
data_dict['results'][0]
new_data = data['results'][0]
new_data

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [94]:
import pandas as pd

def foursquare_data(data):
    # Initialize empty lists to store extracted data
    fsq_id = []
    categories = []
    distance = []
    latitude = []
    longitude = []
    address = []
    country = []
    locality = []

    for result in data['results']:
        fsq_id = result.get('fsq_id', None)
        
         # Use list comprehension to join category names
        category_names = ', '.join([cat['name'] for cat in result['categories']]) # Use list comprehension
        
        dist = result.get('distance', None)
        lat = result['geocodes']['main']['latitude']
        lon = result['geocodes']['main']['longitude']
        addr = result.get('location', {}).get('address', None)
        cntry = result.get('location', {}).get('country', None)
        loc = result.get('location', {}).get('locality', None)
        
        # Append extracted data to respective lists
        fsq_id.append(fsq_id)
        categories.append(category_names)
        distance.append(dist)
        latitude.append(lat)
        longitude.append(lon)
        address.append(addr)
        country.append(cntry)
        locality.append(loc)

    # Create a DataFrame from the extracted data
    data_frame = pd.DataFrame({
        'Foursquare ID': fsq_id,
        'Categories': categories,
        'Distance': distance,
        'Latitude': latitude,
        'Longitude': longitude,
        'Address': address,
        'Country': country,
        'Locality': locality
    })
    
    return data_frame

data_fsq = foursquare_data(data)
print(data_fsq)

               Foursquare ID                                       Categories  \
0   4ef1daed0e617e29c65f76c9                    Café, Coffee Shop, Restaurant   
1   4c595eb067ac0f474df7034c                                  Bar, Restaurant   
2   4b6cb501f964a520de4d2ce3                                         Pizzeria   
3   4bf728165ec320a1b28d86d3          Cafe, Coffee, and Tea House, Restaurant   
4   4b9e2a28f964a520a6ce36e3          Cafe, Coffee, and Tea House, Restaurant   
5   4de9b9701f6e3ddebdba024f                                         Pizzeria   
6   4c76ea765cd6b1f7d3743d94                                       Food Court   
7   4bad451bf964a5203a403be3                                       Restaurant   
8   4e544e9fae60e3d6b7eccc42                       Deli, Fast Food Restaurant   
9   4d8ba6940c4e41bd15fa667f                   Pizzeria, Fast Food Restaurant   
10  4d9d1dc37865a093ef80e17a                                       Restaurant   
11  4dd0499d52b1f8915bc47e93

Put your parsed results into a DataFrame

In [96]:
import pandas as pd
# Create a DataFrame from the extracted data
data_fsq.info()
# Save the DataFrame as a CSV file
data_fsq.to_csv('fsq_data.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Foursquare ID  50 non-null     object 
 1   Categories     50 non-null     object 
 2   Distance       50 non-null     int64  
 3   Latitude       50 non-null     float64
 4   Longitude      50 non-null     float64
 5   Address        50 non-null     object 
 6   Country        50 non-null     object 
 7   Locality       50 non-null     object 
dtypes: float64(2), int64(1), object(5)
memory usage: 3.2+ KB


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [97]:
import requests
import pandas as pd
import os

# yelp api
yelp_api_key = "API KEY"
# Initialize an empty list to store results
yelp_results = []

# Iterate through the rows of the DataFrame
for index, row in df_new.iterrows():
    latitude = row['Latitude']
    longitude = row['Longitude']
    radius = 1000  # You can adjust the radius as needed
    categories = "restaurants"  # You can adjust the category as needed
    limit = 50  # You can adjust the limit as needed

    url = f"https://api.yelp.com/v3/businesses/search"

    params = {
        "latitude": latitude,
        "longitude": longitude,
        "radius": radius,
        "categories": categories,
        "limit": limit
    }

    headers = {
        "Authorization": f"Bearer {yelp_api_key}"
    }
    # Make the API request
    response = requests.get(url, params=params, headers=headers)
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        yelp_data = response.json()
        yelp_results.append(yelp_data)
    else:
        print(f"Failed to retrieve data for latitude: {latitude}, longitude: {longitude}. Status code: {response.status_code}")

# Process and extract relevant information from 'yelp_results' as needed

In [ ]:
yelp_data

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [99]:
import pandas as pd

def extract_yelp_data(yelp_data):
    # Initialize empty lists to store extracted data
    yelp_id = []
    categories = []
    distance = []
    latitude = []
    longitude = []
    address = []
    city = []
    name = []
    rating = []

    businesses = yelp_data.get('businesses', [])

    if not businesses:
        print("No businesses found in the input data.")
        return None

    for business in businesses:
        yelp_id = business.get('id', None)
        category_names = ', '.join([cat['title'] for cat in business.get('categories', [])])
        dist = business.get('distance', None)
        lat = business['coordinates']['latitude']
        lon = business['coordinates']['longitude']
        addr = ', '.join(business.get('location', {}).get('display_address', []))
        city_name = business.get('location', {}).get('city', None)
        business_name = business.get('name', None)
        business_rating = business.get('rating', None)

        # Append extracted data to respective lists
        yelp_ids.append(yelp_id)
        categories.append(category_names)
        distance.append(dist)
        latitude.append(lat)
        longitude.append(lon)
        address.append(addr)
        city.append(city_name)
        name.append(business_name)
        rating.append(business_rating)

    # Create a DataFrame from the extracted data
    data_frame = pd.DataFrame({
        'Yelp ID': yelp_id,
        'Categories': categories,
        'Distance': distance,
        'Latitude': latitude,
        'Longitude': longitude,
        'Address': address,
        'City': city,
        'Name': name,
        'Rating': rating
    })

    return data_frame

                   Yelp ID                                  Categories  \
0   TqMma1y2oMUn7X7CUa326A                           Burgers, American   
1   JS8Jsxj5TFo2g3PAAfpTCQ            Coffee & Tea, Breakfast & Brunch   
2   pcsS_FI0pmQgDjSaGg2ydQ                                  Tapas Bars   
3   1astxknYDdqjS_tuqWA6DA                               Italian, Bars   
4   VD1YD4oALbOUWUspR4_3fQ            Cafes, Coffee & Tea, Hobby Shops   
5   IjgiLzO1Zavr0xarKcyuTw                                       Pizza   
6   PUrIAfvclMBH20rtvjRehA                                  Sandwiches   
7   6Dllc3lnPfr8rXZ-GH1aIw                        Seafood, Steakhouses   
8   ciKjlKZPXRyZTixw7Pi49g                                Fish & Chips   
9   yfOjLtmJifEnl25XK_Cdwg                    Cheesesteaks, Sandwiches   
10  ujYeM5SgbiRuUiwnxbXvQA                                     Italian   
11  yu7r0_gPJq5-Taw8c9QbEw                                       Cafes   
12  kaoPyoYyyaUJhreSDQfycg            

Put your parsed results into a DataFrame

In [100]:
data_yelp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Yelp ID     50 non-null     object 
 1   Categories  50 non-null     object 
 2   Distance    50 non-null     float64
 3   Latitude    50 non-null     float64
 4   Longitude   50 non-null     float64
 5   Address     50 non-null     object 
 6   City        50 non-null     object 
 7   Name        50 non-null     object 
 8   Rating      50 non-null     float64
dtypes: float64(4), object(5)
memory usage: 3.6+ KB


In [101]:
import pandas as pd
yelpdf = pd.DataFrame(data_yelp)
data_yelp.to_csv('yelp_data.csv', index=False)

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Yelp provided more complete data than Foursquare. It had information for ratings, phone number, url, etc. It also located more businesses.

Get the top 10 restaurants according to their rating

In [104]:
# Check if 'businesses' key is present in the Yelp data
if 'businesses' in yelp_data:
    # Extract the list of businesses
    businesses = yelp_data['businesses']

    # Sort businesses based on rating in descending order
    business_rating = sorted(businesses, key=lambda x: x['rating'], reverse=True)

    # Select the top 10 restaurants
    top_10_restaurants = business_rating[:10]

    # Iterate through the top 10 restaurants and print details
    for index, restaurant in enumerate(top_10_restaurants, start=1):
        print(f"Rank {index}:")
        print(f"Name: {restaurant['name']}")
        print(f"Rating: {restaurant['rating']}")
        print(f"Location: {restaurant['location']['address1']}, {restaurant['location']['city']} {restaurant['location']['zip_code']}")
        print(f"Category: {', '.join([category['title'] for category in restaurant['categories']])}")
        print(f"url: {restaurant['url']}")
        print(f"phone: {restaurant['phone']}")
        print("\n")
else:
    print("No business data found in the Yelp response.")


Rank 1:
Name: Mancala Monk Board Game Cafe
Rating: 5.0
Location: 1229 Cannon Street E, Hamilton L8H 1T8
Category: Cafes, Coffee & Tea, Hobby Shops
url: https://www.yelp.com/biz/mancala-monk-board-game-cafe-hamilton?adjust_creative=xCtHNVB74WSlES58SRjTgA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=xCtHNVB74WSlES58SRjTgA
phone: +19053936815


Rank 2:
Name: Ottawa Market
Rating: 5.0
Location: 204 Ottawa St N, Hamilton L8H 3Z5
Category: Florists, Cafes, Grocery
url: https://www.yelp.com/biz/ottawa-market-hamilton-2?adjust_creative=xCtHNVB74WSlES58SRjTgA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=xCtHNVB74WSlES58SRjTgA
phone: 


Rank 3:
Name: Das Schnitzel
Rating: 5.0
Location: 131 Ottawa Street N, Hamilton L8L 1Y5
Category: Soup, Burgers, Sandwiches
url: https://www.yelp.com/biz/das-schnitzel-hamilton?adjust_creative=xCtHNVB74WSlES58SRjTgA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=xCtHNVB74WSlES58SRjTgA
phone: